In [ ]:
""" Задачи """

Необходимо сравнить мощности тестов с разной долей удаляемых данных. Используйте данные о времени работы бэкенда df_web_logs.csv в период с 2022-03-01 по 2022-03-08. 

Уровень значимости — 0.05. 

Размеры групп — 1000 человек (размер выборок будет больше, так как на одного человека приходится много значений). 

Проверяем гипотезу о равенстве средних с помощью теста Стьюдента. Ожидаемый эффект — увеличение времени обработки на 1%. Эффект в синтетических А/В-тестах добавляем умножением на константу.

In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from scipy import stats
from tqdm.notebook import tqdm

In [2]:
def read_file(file_name):
    if not os.path.exists(file_name):
        raise FileNotFoundError(f'Файл {file_name} не найден')
    return pd.read_csv(file_name)

In [3]:
df_logs = read_file('df_web_logs.csv')

df_logs['date'] = pd.to_datetime(df_logs['date'])
date_start = datetime(2022, 3, 1)
date_end = datetime(2022, 3, 8)

In [4]:
df_metrics = df_logs[(df_logs['date'] >= date_start) & (df_logs['date'] < date_end)][['user_id', 'load_time']].reset_index(drop=True)

In [5]:
df_metrics.head()

,user_id,load_time
0,434cf2,69.8
1,80fa93,86.3
2,434cf2,58.0
3,a0aaab,85.2
4,a22f92,92.5


In [6]:
df_users = df_metrics['user_id'].unique()

In [7]:
sample_size = 1000
effect = 0.01
alpha = 0.05
quantiles = (0.0001, 0.001, 0.01, 0.05, 0.1,)
quantile2errors = {q: [] for q in quantiles}

In [9]:
for _ in range(1000):
    a_users, b_users = np.random.choice(df_users, (2, sample_size,), False)
    a_values = df_metrics.loc[df_metrics['user_id'].isin(a_users), 'load_time'].values
    b_values = (
        df_metrics.loc[df_metrics['user_id'].isin(b_users), 'load_time'].values
        * (1 + effect)
    )
    for q in quantiles:
        a_values_filtered = a_values[
            (a_values > np.quantile(a_values, q))
            & (a_values < np.quantile(a_values, 1-q))
        ]
        b_values_filtered = b_values[
            (b_values > np.quantile(b_values, q))
            & (b_values < np.quantile(b_values, 1-q))
        ]
        pvalue = stats.ttest_ind(a_values_filtered, b_values_filtered).pvalue
        quantile2errors[q].append(pvalue > alpha)
        

In [12]:
def process_results(quantile2errors):
    """Обрабатываем данные и выводим результат."""
    # список из элементов: номер варианта, квантиль, доля ошибок, список ошибок).
    data = [
        (idx+1, quantile, np.mean(errors), errors,)
        for idx, (quantile, errors) in enumerate(quantile2errors.items())
    ]
    # сортируем по доле ошибок
    data.sort(key=lambda x: x[2])

    # проверим, что оценки мощности значимо отличаются друг от друга
    print(f'idx={data[0][0]} quantile={data[0][1]}, power={1-data[0][2]:0.3f}')
    for i in range(1, len(data)):
        pvalue = stats.ttest_ind(data[i][3], data[i-1][3]).pvalue
        if pvalue < 0.05:
            msg = f'pvalue={pvalue:0.4f}, оценка мощности значимо отличается от предыдущей'
        else:
            msg = f'pvalue={pvalue:0.4f}, оценка мощности не отличается значимо от предыдущей'
        print(f'idx={data[i][0]} quantile={data[i][1]}, power={1-data[i][2]:0.3f}, {msg}')
    print('answer:', ''.join([str(x[0]) for x in data]))

process_results(quantile2errors)

idx=5 quantile=0.1, power=0.970
idx=4 quantile=0.05, power=0.963, pvalue=0.3443, оценка мощности не отличается значимо от предыдущей
idx=3 quantile=0.01, power=0.933, pvalue=0.0016, оценка мощности значимо отличается от предыдущей
idx=2 quantile=0.001, power=0.340, pvalue=0.0000, оценка мощности значимо отличается от предыдущей
idx=1 quantile=0.0001, power=0.089, pvalue=0.0000, оценка мощности значимо отличается от предыдущей
answer: 54321
